In [ ]:
pwd

# 1 Intro

Notebook dédié à l'entrainement du modèle de ML
- sur base application_train (train_V1.csv)
- dummy classifier
- modèles de classification: Regression Logistique, Random Forest, LGBM, XGBooost
- transformation des variables catégorielles via OneHotEncoder puis via booléen (quand possible)
- gestion de classes non équilibrées(RandomUnderSampler)


## 1.1. Import

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import chardet
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.impute import KNNImputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import class_weight
from imblearn.under_sampling import RandomUnderSampler, CondensedNearestNeighbour, ClusterCentroids,EditedNearestNeighbours, AllKNN, TomekLinks
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as Pipeline_imb
import lightgbm as lgb
from lightgbm import LGBMClassifier
from lightgbm import plot_importance
from xgboost import XGBClassifier
import xgboost as xgb
import mlflow
from mlflow import log_metric, log_param, log_artifacts, log_metrics
from mlflow.models import infer_signature
from mlflow.sklearn import log_model
import os
import tempfile
import datetime
from datetime import datetime as dt
import warnings
warnings.filterwarnings('ignore')
import pickle
import yellowbrick
from yellowbrick import ROCAUC

## 1.2 Fonctions utiles / set-up

In [ ]:
from src.fonctions import *

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
DISPLAY=True

"""If Dispaly == False then all cell starting starting with 'If Display=True' won't be ran.
Else they will

This short function allows to by-pass instructions which are long to run (ex MSNO on huge dataset or sns.pairplot)"""

# 2 Data

## 2.1 Import and display

In [ ]:
train_V1=pd.read_csv('../train_V1.csv')

In [ ]:
train_V2=pd.read_csv('../train_V2.csv')

In [ ]:
train_V1.shape

In [ ]:
train_V1.head()

Select only 20% du dataset pour accelerer divers fits ci dessous

In [ ]:
train_V1=train_V1.sample(frac=0.2)
train_V1.shape

# 3 Phase 0: MachineLearning / Baseline
- Sur base Application Train
- Dummy Classifier

## 3.1. Séparation du jeu de données (features: X / target:y)
 et suppression de colonnes inutiles

In [ ]:
X=train_V1.drop(['SK_ID_CURR','TARGET','OWN_CAR_AGE','NAME_HOUSING_TYPE','AMT_GOODS_PRICE','active_client','relationship'],axis=1)
X.shape

In [ ]:
y=train_V1['TARGET']
y.shape

In [ ]:
num_feat=X.select_dtypes(exclude=object).columns.to_list()
num_feat

In [ ]:
cat_feat=X.select_dtypes(include=object).columns.to_list()
cat_feat

## 3.2 Train-test split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
print('Train set:',X_train.shape)
print('Test set:',X_test.shape)
print('Train target:',y_train.shape)
print('Test target:',y_test.shape)

## 3.3 Entrainement du DummyClassifier

In [ ]:
dc=DummyClassifier()

In [ ]:
dc.fit(X_train,y_train)

In [ ]:
dc_pred=dc.predict(X_test)

In [ ]:
acc=accuracy_score(y_test, dc_pred)
acc

In [ ]:
error=1-dc.score(X_test,y_test)
error

In [ ]:
1-acc


In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test,dc_pred)).plot()

In [ ]:
print_score(y_test, dc_pred)

## 3.5 Log MLFlow

In [ ]:
params={}

In [ ]:
metrics={"accuracy": accuracy_score(y_test, dc_pred), "ROC AUC": roc_auc_score(y_test, dc_pred)}

In [ ]:
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

In [ ]:
mlflow.set_experiment('credit_default_learning')

In [ ]:
with mlflow.start_run():
       
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metrics(metrics)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "DummyClassifier for credit default")

    # Infer the model signature
    #signature = infer_signature(X_train, dc.predict(X_train))

    # Log the model

    mlflow.sklearn.log_model(
        sk_model=dc,
        artifact_path="pret_a_depenser",
        #signature=signature,
        input_example=X_train.iloc[:10],
        registered_model_name="tracking-dummy_classifier",
    )


# 4 V1: Autres modèles et pipeline sur base train_V1

Pour le pipeline:
- sampler: RandomUnderSampler() (imblearn) 
- transformer: OHE pour cat_feat / log pour feat AMT
- imputer: SimpleImputer('median')
- scaler: StandardScaler()
- estimator = DummyClassifier(), LogReg(), RF(),LightGBM, XGBoost

Pour les metrics: priotité au ROC aUC
récupération de F1, recall, accuracy, precision

Et 'speci: fonction créée pour calculer spécificité (TN / (TN+FP)) qui doit être le + grand possible 


#### Transformer

In [ ]:
log_fct = FunctionTransformer(np.log1p)

In [ ]:
transformer= ColumnTransformer(transformers=
                               [
                               ('OneHot',OneHotEncoder(sparse=False,handle_unknown='ignore'),cat_feat),
                               ('num',log_fct,['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY'])                            
                               ],
                               remainder='passthrough'
                             )
transformer

#### Pipeline

In [ ]:
pipeline=Pipeline_imb([
    ('sampler',RandomUnderSampler()),
    ('transformer',transformer),
    ('imputer',SimpleImputer()),
    ('scaler',StandardScaler()),    
    ('estimator',LogisticRegression())
])
pipeline

#### Paramgrid

- Avec ou sans scaler
- Avec ou sans sampler (ici RandomUnderSampler() car reste plus de 20000 individus de chaque catégorie (A essayer avec d'autres techniques: SMOTE, oversmapling, class_weight...)
- différents estimateurs: LogisticRegression, RandomForest, XGBoost, LightGBM

In [ ]:
param_grid={
    'sampler':[RandomUnderSampler(), 'passthrough'],
    'scaler':[StandardScaler(),'passthrough'],    
    'estimator':[LogisticRegression(),RandomForestClassifier(),LGBMClassifier(),XGBClassifier()]
}

#### GridsearchCV

In [ ]:
scoring = {
        'roc_auc': 'roc_auc',
        'accuracy': 'accuracy',
        'f1': 'f1',
        'recall':'recall',
        'precision':'precision',
        'speci': (make_scorer(my_specificity_score,))# greater_is_better=False,))
}

In [ ]:
grid= GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=5,
    #scoring=('roc_auc','accuracy','f1','recall','precision'),
    scoring=scoring,
    n_jobs=-1,
    verbose=2,
    return_train_score=True,
    refit='roc_auc')

In [ ]:
if DISPLAY:
    grid.fit(X,y)

In [ ]:
if DISPLAY:
    res=result(grid, log_target=1)
    res.sort_values(by='mean_test_roc_auc',ascending=False).head(5)

In [ ]:
RES=pd.DataFrame()

In [ ]:
RES=pd.concat([RES, res],axis=0, ignore_index=True)

In [ ]:
RES.head(10)

#### Log Mlflow

In [ ]:
log_results(grid, experiment_name='credit_default_learning',model_name='1st try_with_log_V1')

#### Même chose sans log

In [ ]:
transformer2=ColumnTransformer(transformers=
                               [
                               ('OneHot',OneHotEncoder(sparse=False,handle_unknown='ignore'),cat_feat),
                               ],
                               remainder='passthrough'
                             )
transformer2

In [ ]:
pipeline2=Pipeline_imb([
    ('sampler',RandomUnderSampler(replacement=True)),
    ('transformer',transformer2),
    ('imputer',SimpleImputer()),
    ('scaler',StandardScaler()),
    ('estimator',LogisticRegression())
])
pipeline2

In [ ]:
grid= GridSearchCV(
    pipeline2,
    param_grid=param_grid,
    cv=5,
    scoring=scoring,
    n_jobs=-1,
    verbose=2,
    return_train_score=True,
    refit='roc_auc')

In [ ]:
if DISPLAY:
    grid.fit(X,y)

In [ ]:
res=result(grid, log_target=0)
res.sort_values(by='mean_test_roc_auc',ascending=False).head(5)

In [ ]:
RES=pd.concat([RES, res],axis=0, ignore_index=True)
RES.head(10)

In [ ]:
grid.best_params_

#### MLflow

In [ ]:
log_results(grid,experiment_name='credit_default_learning',model_name='P7_sans_log_V1')

#### Remplaçons certaines variables par leur booléen (Income type et Family status)

In [ ]:
train_V1.columns.to_list()

In [ ]:
X=train_V1.drop(['SK_ID_CURR','TARGET','OWN_CAR_AGE','NAME_HOUSING_TYPE','AMT_GOODS_PRICE','NAME_INCOME_TYPE','NAME_FAMILY_STATUS'],axis=1)
X.shape

In [ ]:
y.shape

In [ ]:
cat_feat=X.select_dtypes(include=object).columns.to_list()
cat_feat

In [ ]:
num_feat

#### avec passage au log (transformer)

In [ ]:
transformer= ColumnTransformer(transformers=
                               [
                               ('OneHot',OneHotEncoder(sparse=False,handle_unknown='ignore'),cat_feat),
                               ('num',log_fct,['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY'])
                               ],
                               remainder='passthrough'
                             )
transformer

In [ ]:
pipeline=Pipeline_imb([
    ('sampler',RandomUnderSampler(replacement=True)),
    ('transformer',transformer),
    ('imputer',SimpleImputer()),
    ('scaler',StandardScaler()),
    ('estimator',DummyClassifier())
])
pipeline

In [ ]:
grid= GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=5,
    scoring=scoring,
    n_jobs=-1,
    verbose=2,
    return_train_score=True,
    refit='roc_auc')

In [ ]:
if DISPLAY:
    grid.fit(X,y)

In [ ]:
res=result(grid, log_target=1)
res.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

In [ ]:
RES=pd.concat([RES, res],axis=0, ignore_index=True)

In [ ]:
RES.sort_values(by='mean_test_roc_auc',ascending=False).head(20)

#### MLflow

In [ ]:
log_results(grid,experiment_name='credit_default_learning',model_name='P7_bool_with_log_V1')

#### sans passage au log (transformer2)

In [ ]:
transformer2=ColumnTransformer(transformers=
                               [
                               ('OneHot',OneHotEncoder(sparse=False,handle_unknown='ignore'),cat_feat),
                               ],
                               remainder='passthrough'
                             )
transformer2

In [ ]:
pipeline2=Pipeline_imb([
    ('sampler',RandomUnderSampler(replacement=True)),
    ('transformer',transformer2),
    ('imputer',SimpleImputer()),
    ('scaler',StandardScaler()),
    ('estimator',DummyClassifier())
])
pipeline2

In [ ]:
grid= GridSearchCV(
    pipeline2,
    param_grid=param_grid,
    cv=5,
    scoring=scoring,
    n_jobs=-1,
    verbose=2,
    return_train_score=True,
    refit='roc_auc')

In [ ]:
if DISPLAY:
    grid.fit(X,y)

In [ ]:
res=result(grid, log_target=0)
res.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

In [ ]:
RES=pd.concat([RES, res],axis=0, ignore_index=True)

#### MLflow

In [ ]:
log_results(grid,experiment_name='credit_default_learning',model_name='P7_bool_sans_log_V1')

#### Sans variable catégorielle

In [ ]:
train_V1.columns.to_list()

In [ ]:
X=train_V1.drop(['SK_ID_CURR','TARGET','OWN_CAR_AGE','NAME_HOUSING_TYPE','AMT_GOODS_PRICE','NAME_INCOME_TYPE','NAME_FAMILY_STATUS','NAME_EDUCATION_TYPE'],axis=1)
X.shape

In [ ]:
cat_feat=X.select_dtypes(include=object).columns.to_list()
cat_feat

In [ ]:
transformer3= ColumnTransformer(transformers=
                               [
                               ('num',log_fct,['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY'])
                               ]
                             )
transformer3

In [ ]:
pipeline3=Pipeline_imb([
    ('sampler',RandomUnderSampler(replacement=True)),
    ('transformer',transformer3),
    ('imputer',SimpleImputer()),
    ('scaler',StandardScaler()),
    ('estimator',DummyClassifier())
])
pipeline3

In [ ]:
param_grid={
    'sampler':[RandomUnderSampler(replacement=True), 'passthrough'],'transformer':[transformer3,'passthrough'],
    'scaler':[StandardScaler(),'passthrough'], 
    'estimator':[LogisticRegression(),RandomForestClassifier(),LGBMClassifier(),XGBClassifier()]
}

In [ ]:
grid= GridSearchCV(
    pipeline3,
    param_grid=param_grid,
    cv=5,
    scoring=scoring,
    n_jobs=-1,
    verbose=2,
    return_train_score=True,
    refit='roc_auc')

In [ ]:
if DISPLAY:
    grid.fit(X,y)

In [ ]:
res=result(grid, log_target=0)
res.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

In [ ]:
RES=pd.concat([RES, res],axis=0, ignore_index=True)

In [ ]:
RES.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

In [ ]:
RES.to_csv('../RES.csv', index =False)

In [ ]:
log_results(grid,experiment_name='credit_default_learning',model_name='P7_no_cat_feat_V1')

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_params_

In [ ]:
RES.sort_values(by='mean_test_roc_auc',ascending=False).head(20)

## 1eres conclusions

- résultats meilleurs avec LogReg() et LGBM
- LogReg() bcp + raide à entrainer
- Avec sampler. Sans sampling, spécificité est très bonne mais recall proche de 0. Attention à ce score: sans sampling TN+TP est 10 fois plus important qu'avec testing.
    => A omparer avec SMOTE
- sans var catégorielles (ie sans passage par OHE) => à revérifier suite à ajout de reamainder = 'passthrough'
- avec scaler
- avec log
- avec sampler (ie avec equilibrage des classes)

est: LogReg()
ROC_AUC: 0,7388
Recall: 0,67
Speci: 0.68

=> Partir sur ces bases pour amélioratiosn des hyperparamètres

In [ ]:
RES.sort_values(by='mean_test_f1',ascending=False).head(10)

# 5 Sur base train_V2 

Prise en compte de la V2 du features engiennering (train_V2.csv):
- ajout de features inspirées du kernel disponible ne ressource du projetavec experimentatin de critères dans Param_grid
- ajout de features issues des datasets annexes (bureau et previous applications)

Par ailleurs essai avec:
- imputer: KNNImputer()
- sampler: SMOTE()

## 5.1 Import 

In [ ]:
train_V2=pd.read_csv('../train_V2.csv')

In [ ]:
train_V2.shape

Select only 20% du dataset pour accelerer divers fits ci dessous

In [ ]:
train_V2=train_V2.sample(frac=0.2)
train_V2.shape

## 5.2 Séparation du jeu de données (features: X / target:y)
et suppression de colonnes inutiles

In [ ]:
X=train_V2.drop(['SK_ID_CURR','TARGET','OWN_CAR_AGE','NAME_HOUSING_TYPE','AMT_GOODS_PRICE','NAME_INCOME_TYPE','NAME_FAMILY_STATUS','NAME_EDUCATION_TYPE'],axis=1)
X.shape

In [ ]:
y=train_V2['TARGET']
y.shape

In [ ]:
num_feat=X.select_dtypes(exclude=object).columns.to_list()
num_feat

In [ ]:
cat_feat=X.select_dtypes(include=object).columns.to_list()
cat_feat

## 5.3 Sans variable catégorielle

In [ ]:
transformer= ColumnTransformer(transformers=
                               [
                               ('num',log_fct,['AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY'])
                               ]
                             )
transformer

In [ ]:
pipeline=Pipeline_imb([
    ('transformer',transformer),
    ('imputer',SimpleImputer()),
    ('sampler',RandomUnderSampler()),
    ('scaler',StandardScaler()),
    ('estimator',LogisticRegression())
])
pipeline

In [ ]:
param_grid={
    'sampler':[ RandomUnderSampler(), SMOTE()],
    'transformer':[transformer,'passthrough'],
    'scaler':[StandardScaler(),'passthrough'], 
    'estimator':[LogisticRegression(),LGBMClassifier()]
}

In [ ]:
grid= GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=5,
    scoring=scoring,
    n_jobs=-1,
    verbose=2,
    return_train_score=True,
    refit='roc_auc')

In [ ]:
if DISPLAY:
    grid.fit(X,y)

In [ ]:
res=result(grid,features='train_V2')
res.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

In [ ]:
RES=pd.concat([RES, res],axis=0, ignore_index=True)
RES.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

In [ ]:
RES.to_csv('../RES.csv', index =False)

Résultats légerement meilleurs sur train_V2

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_params_

### Log MLflow

In [ ]:
log_results(grid,experiment_name='credit_default_learning',model_name='P7_no_cat_feat_V2',tags={'V2':'train_V2'})

## 5.4 Essai divers under samplers

Perimètre très réduit (1%; sur 20% dataset, entrainement très long)

In [ ]:
tmp=train_V2.sample(frac=0.05)
tmp.shape

In [ ]:
X=tmp.drop(['SK_ID_CURR','TARGET','OWN_CAR_AGE','NAME_HOUSING_TYPE','AMT_GOODS_PRICE','NAME_INCOME_TYPE','NAME_FAMILY_STATUS','NAME_EDUCATION_TYPE'],axis=1)
X.shape

In [ ]:
y=tmp['TARGET']
y.shape

In [ ]:
pipeline=Pipeline_imb([
    #('transformer',transformer3),
    ('imputer',SimpleImputer()),
    ('sampler',RandomUnderSampler()),
    ('scaler',StandardScaler()),
    ('estimator',LogisticRegression())
])
pipeline

In [ ]:
param_grid={
    'sampler':[RandomUnderSampler(),ClusterCentroids(),EditedNearestNeighbours(),TomekLinks()], #AllKNN()
    #'transformer':[transformer3,'passthrough'],
    'scaler':[StandardScaler()], 
    'estimator':[LogisticRegression(),LGBMClassifier()]
}

In [ ]:
grid= GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=5,
    scoring=scoring,
    n_jobs=-1,
    verbose=2,
    return_train_score=True,
    refit='roc_auc')

In [ ]:
if DISPLAY:
    grid.fit(X,y)

In [ ]:
res=result(grid,transf_feat='test under sample',features='train_V2')
res.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

Pas de meilleurs résultats avec autres under samplers (et entrainements beaucoup plus longs)

In [ ]:
RES=pd.concat([RES, res],axis=0, ignore_index=True)
RES.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

In [ ]:
RES.to_csv('../RES.csv', index =False)

Log MLflow

In [ ]:
log_results(grid,experiment_name='credit_default_learning',model_name='P7_no_cat_feat_reduced_V2',tags={'V2':'reduced_train_V2'})

## 5.5 Essai KNNImputer

Perimètre très réduit (1%; sur 20% dataset, plantage systématique)

In [ ]:
pipeline=Pipeline_imb([
    #('transformer',transformer3),
    ('imputer',KNNImputer()),
    ('sampler',RandomUnderSampler()),
    ('scaler',StandardScaler()),
    ('estimator',LogisticRegression())
    ])
pipeline

In [ ]:
param_grid={
    'sampler':[RandomUnderSampler()],
    'scaler':[StandardScaler(),], 
    'estimator':[LogisticRegression(),LGBMClassifier(),]
}

In [ ]:
grid= GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=5,
    scoring=scoring,
    n_jobs=-1,
    verbose=2,
    return_train_score=True,
    refit='roc_auc')

In [ ]:
grid.fit(X,y)

In [ ]:
res=result(grid,transf_feat='KNNImputer',features='train_V2')
res.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

Pas de meilleurs résultats avec KNN

In [ ]:
RES=pd.concat([RES, res],axis=0, ignore_index=True)
RES.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

In [ ]:
RES.to_csv('../RES.csv', index =False)

Log MLflow

In [ ]:
log_results(grid,experiment_name='credit_default_learning',model_name='P7_no_cat_feat_red_V2_KNNImp',tags={'V2':'reduced_train_V2'})

# 6 Améliorations hyperparamètres:
- sur best model LogReg()
- sur best model LGBM()

In [ ]:
X=train_V2.drop(['SK_ID_CURR','TARGET','OWN_CAR_AGE','NAME_HOUSING_TYPE','AMT_GOODS_PRICE','NAME_INCOME_TYPE','NAME_FAMILY_STATUS','NAME_EDUCATION_TYPE'],axis=1)
X.shape

In [ ]:
y=train_V2['TARGET']
y.shape

## 6.1 Sur LogReg()

In [ ]:
best_logreg=Pipeline_imb([
    ('sampler',RandomUnderSampler()),
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler()),
    ('estimator',LogisticRegression())
])
best_logreg

In [ ]:
param_grid={
    'sampler':[ RandomUnderSampler()],
    'imputer':[SimpleImputer(strategy='median'),SimpleImputer(strategy='mean')],
    'scaler':[StandardScaler()],
    'estimator':[LogisticRegression()],
    'estimator__solver':['newton-cg','lbfgs','sag', 'saga'],
    'estimator__penalty':['none', 'elasticnet', 'l1', 'l2'],
    'estimator__C':[0.001, 0.01, 0.1, 1, 10, 100]    
}

In [ ]:
grid= GridSearchCV(
    best_logreg,
    param_grid=param_grid,
    cv=5,
    scoring=scoring,
    n_jobs=-1,
    verbose=2,
    return_train_score=True,
    refit='roc_auc')

In [ ]:
grid.fit(X,y)

In [ ]:
res=result(grid,features='train_V2')
res.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

In [ ]:
RES=pd.concat([RES, res],axis=0, ignore_index=True)
RES.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

In [ ]:
RES.to_csv('../RES.csv', index =False)

Résultats équivalents. Après qlq runs amélioration du best score 0.7414 à 0.7443. 

J'arrête ici sur l'amélioration des hyperparamètres. Les résultats sont presque identiques et à chaque run ce sont des meilleurs hyperparamètres différents

## 6.2 Sur LGBM()

In [ ]:
best_LGBM=Pipeline_imb([
    ('sampler',RandomUnderSampler()),
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler()),
    ('estimator',LGBMClassifier())
])
best_LGBM

In [ ]:
param_grid={
    'sampler':[ RandomUnderSampler()],
    'imputer':[SimpleImputer(strategy='median'),SimpleImputer(strategy='mean')],
    'scaler':[StandardScaler()],
    'estimator':[LGBMClassifier()],
    'estimator__num_leaves':[20,25,31],
    'estimator__max_depth':[3],#,6],
    #'estimator__learning_rate':[0.001, 0.01, 0.1],
    'estimator__min_data_in_leaf':[500,750,1000]
}

In [ ]:
grid= GridSearchCV(
    best_LGBM,
    param_grid=param_grid,
    cv=5,
    scoring=scoring,
    n_jobs=-1,
    verbose=2,
    return_train_score=True,
    refit='roc_auc')

In [ ]:
grid.fit(X,y)

In [ ]:
res=result(grid,features='train_V2')
res.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

Après qlq essais on atteint 0.75

In [ ]:
RES=pd.concat([RES, res],axis=0, ignore_index=True)
RES.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

Log Mlflow

In [ ]:
log_results(grid,experiment_name='credit_default_learning',model_name='P7_no_cat_feat_V2',tags={'V2':'train_V2'})

# 7 Amélioration modèle avec entrienement uniquement sur lignes sans NaN sur EXT_SOURCE_1, 2 et 3

- Imputation par SimplImputer de 3 features les plus importantes me semble peu satisfaisant
- Par ailleur nb d'invididus important, on peut se permettre d'exclure les individus avec NaN et réduire le sampling initial 


## 7.1 Import

In [ ]:
train_V2=pd.read_csv('../train_V2.csv')

In [ ]:
train_V2= train_V2.drop(['SK_ID_CURR','OWN_CAR_AGE','NAME_HOUSING_TYPE','AMT_GOODS_PRICE','NAME_INCOME_TYPE','NAME_FAMILY_STATUS','NAME_EDUCATION_TYPE'],axis=1)

In [ ]:
train_V2.isna().sum()

Suppression des lignes avec NAN

In [ ]:
train_V2=train_V2.drop(train_V2.loc[(train_V2['EXT_SOURCE_1'].isna())|(train_V2['EXT_SOURCE_2'].isna())|(train_V2['EXT_SOURCE_3'].isna())].index).reset_index(drop=True)
train_V2.isna().mean().sort_values(ascending=False).round(2)

In [ ]:
train_V2.shape

In [ ]:
train_V2['TARGET'].value_counts(normalize=True)

Distribution dansla target différente avec et sans NaN

## 7.2 Séparation du jeu de données (features: X / target:y)

In [ ]:
X=train_V2.drop(['TARGET'],axis=1)
X.shape

In [ ]:
y=train_V2['TARGET']
y.shape

## 7.3 Entrainemet

In [ ]:
best_LGBM=Pipeline_imb([
    ('sampler',RandomUnderSampler()),
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler()),
    ('estimator',LGBMClassifier())
])
best_LGBM

In [ ]:
param_grid={
    'sampler':[ RandomUnderSampler()],
    #'imputer':[SimpleImputer(strategy='median'),SimpleImputer(strategy='mean')],
    'scaler':[StandardScaler()],
    'estimator':[LGBMClassifier()],
    'estimator__num_leaves':[20,25,31],
    'estimator__max_depth':[3,6],
    'estimator__learning_rate':[0.001, 0.01, 0.1],
    'estimator__min_data_in_leaf':[500,750,1000]
}

In [ ]:
scoring={'roc_auc': 'roc_auc',
 'accuracy': 'accuracy',
 'f1': 'f1',
 'recall': 'recall',
 'precision': 'precision',
 'speci': make_scorer(my_specificity_score)}

In [ ]:
grid= GridSearchCV(
    best_LGBM,
    param_grid=param_grid,
    cv=10,
    scoring=scoring,
    n_jobs=-1,
    verbose=2,
    return_train_score=True,
    refit='roc_auc')

In [ ]:
grid.fit(X,y)

In [ ]:
res=result(grid,features='EXT_SRC/no_NaN')
res.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

In [ ]:
RES=pd.read_csv('../RES.csv')

In [ ]:
RES=pd.concat([RES, res],axis=0, ignore_index=True)
RES.sort_values(by='mean_test_roc_auc',ascending=False).head(10)

In [ ]:
RES.to_csv('../RES.csv', index =False)

In [ ]:
grid.best_estimator_

In [ ]:
grid.best_params_

In [ ]:
grid.best_score_

## LogMlflow

In [ ]:
log_results(grid, experiment_name='credit_default_learning',model_name='train_V2_NoNaN on EXT_SRC')

# 8 Meilleurs modèles et représentation graphique 

 - sur l'ensemble du dataset
 - y_pred vs y_true / matrice de confusion

## 8.1 Recupération train_V2 et train_test split

In [ ]:
train=pd.read_csv('../train_V2.csv')
train.shape

In [ ]:
X=train.drop(['SK_ID_CURR','TARGET','OWN_CAR_AGE','NAME_HOUSING_TYPE','AMT_GOODS_PRICE','NAME_INCOME_TYPE','NAME_FAMILY_STATUS','NAME_EDUCATION_TYPE'],axis=1)
X.shape

In [ ]:
y=train['TARGET']
y.shape

Train test split

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
print('Train set:',X_train.shape)
print('Test set:',X_test.shape)
print('Train target:',y_train.shape)
print('Test target:',y_test.shape)

## 8.2 Best model (LightGBM)

In [ ]:
grid.best_estimator_

In [ ]:
best_LGBM=Pipeline_imb([
    ('sampler',RandomUnderSampler()),
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler()),
    ('estimator',LGBMClassifier(max_depth=6, min_data_in_leaf=500,
                                num_leaves=25))
])
best_LGBM

In [ ]:
best_LGBM.fit(X_train,y_train)

In [ ]:
lgbm_predict=best_LGBM.predict(X_test)

In [ ]:
print_score(y_test, lgbm_predict)

In [ ]:
my_specificity_score(y_test,lgbm_predict)

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test,lgbm_predict)).plot(values_format='d')


In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test,lgbm_predict)).plot()

In [ ]:
from yellowbrick.classifier.rocauc import roc_auc
roc_auc(best_LGBM, X_train, y_train, X_test=X_test, y_test=y_test, classes=['Crédit remboursé', 'Crédit non remboursé'])

In [ ]:
from yellowbrick.classifier import PrecisionRecallCurve
viz = PrecisionRecallCurve(best_LGBM)
viz.fit(X_train, y_train)
viz.score(X_test, y_test)
viz.show()

## 8.3 Best LogReg

In [ ]:
best_logreg=Pipeline_imb([
    ('sampler',RandomUnderSampler()),
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler()),
    ('estimator',LogisticRegression(C=10, penalty='none'))
])
best_logreg


In [ ]:
best_logreg.fit(X_train,y_train)

In [ ]:
logreg_predict=best_logreg.predict(X_test)

In [ ]:
print_score(y_test, logreg_predict)

In [ ]:
my_specificity_score(y_test, logreg_predict)

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test,logreg_predict)).plot()

## 8.4 Sauvegarde de best_lgbm au format pkl

Pour utilisation dans API

In [ ]:
pickle_out = open("best_lgbm2.pkl","wb")
pickle.dump(best_LGBM, pickle_out)
pickle_out.close()

In [ ]:
bm= open("best_lgbm2.pkl","rb")
best_LGBM=pd.read_pickle(bm)
best_LGBM

In [ ]:
### Create a Pickle file using serialization for API (cf part 7, done now in order to get right best_model)

#pickle_out = open("classifier.pkl","wb")
#pickle.dump(best_model, pickle_out)
#pickle_out.close()

# 9 Optimisation du seuil optimal de proba

En considérant que les pertes engendrées par un crédit non remboursé peuvent être diminuées par la revente du bien financé, alors on peut augmenter le suiel à partir duquel l'EC refuse l'octroi du crédit

Quels est ce seuil?

Plusieurs étapes pour y parvenir:

-  Plusieurs essais de défintion du gain ou de la perte par crédit en fonction de y-pred, d'un taux d'interet imaginaire et d'un taux de récuparation appliqué au prix du bien financé,
si le crédit n'est pas remboursé.
=> calcul_gain_gross
- Ajout de la variable seuil 
=> calcul_gain_proba_unit
- Puis fonction permettant d'atblir une grille seuil / gain total pour prêt à dépenser
=> calcul_seuil_optimal

Plusieurs contraintes

- AMNT_GOOD_PRICE nécessaire dans cette fonction mais non utilisée pour prédiction (essai avec drop dans ColumnsTransformer ==> KO)
- Or prédiction nécessaire
- Mais prédiction se basant sur BEST_LGBM qui contient un sampler (RandomUnderSampler) qui diminue la taille du dataset entrainé
=> nécessité de décomposer le modèle (sampler, imputer et scaler avec AMNT_GOOD_PRICE), estimator sans AMNT_GOOD_PRICE 
- Par ailleurs en passant le dataset dans le scaler dans Best_LGBM, les montants deviennent inexploitable
- Le taux d'interet n'est pas connu => à défaulter
- nb d'annuités inconnu => forcé dans un 1er temps (montant du crédit / montant de l'annuité) puis abandonné
- Si un crédit n'est pas remboursé nous ne savons pas si une partie a été remboursée ou non => je considère que rien n'a été remboursé (= perte de tout le capital prêté)

## 9.1 Décomposition et reconstruction de best_LGBM

Sur base Train_V2

In [ ]:
train_V2.shape

Dans X_tmp, j'inclus AMT_GOODS_PRICE

In [ ]:
X_tmp=train_V2.drop(['TARGET','OWN_CAR_AGE','NAME_HOUSING_TYPE','NAME_INCOME_TYPE','NAME_FAMILY_STATUS','NAME_EDUCATION_TYPE'],axis=1)
X_tmp.shape

In [ ]:
y=train_V2['TARGET']
y.shape

Train test split sur base X_tmp

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X_tmp,y,test_size=0.2)

In [ ]:
print('Train set:',X_train.shape)
print('Test set:',X_test.shape)
print('Train target:',y_train.shape)
print('Test target:',y_test.shape)

In [ ]:
best_LGBM=Pipeline_imb([
    ('sampler',RandomUnderSampler()),
    ('imputer',SimpleImputer(strategy='median')),
    ('scaler',StandardScaler()),
    ('estimator',LGBMClassifier(max_depth=6, min_data_in_leaf=500,
                                num_leaves=25))
])
best_LGBM

Decomposition du modèle avec fonction créée

In [ ]:
def decomposition_modele(modele, X_train=X_train,X_test=X_test,y_train=y_train,):
    '''
    
    ##### Mettre à jour fonctions.py API_V2 #####    
    
    Fonction permettant de décomposer un modèle (pipeline) imbalanced ou non afin de dérouler les différents étapes de preprocessing sans la partie estimator.
    Le but étant de pouvoir gérer la feature importance en conservant le nom des colonnes de X_train
    
    ATTENTION: cette fonction ne fonctionne qu'avec un type précis de pipeline davec les étapes suivantes:
    - le 1er step (non obligatoire) de UnderSampling (uniquement RandomUnderSampler())
    - un ou plusieurs steps de transformation (scaler/imputer)
    - le dernier correspo,d à l'estimateur
    
    Arguments:
    - modele: de type pipeline
    - X_train
    - X_test
    - y_train (necessaire pour le resampling)
    '''
        
    # Si modèle imbalanced:
    
    if str(modele[0]) == 'RandomUnderSampler()':
        
        #Sampling
        
        X_tr, y_train_rus=modele[0].fit_resample(X_train,y_train)
    
        # Preprocessing X_train
    
        X_tr=modele[1:-1].fit_transform(X_tr)
        X_tr_transf=pd.DataFrame(X_tr, columns=X_train.columns)
    
        # Preprocessing X_test
    
        #X_te_transf=modele[1:-1].fit_transform(X_test)
        X_te_transf=modele[1:-1].transform(X_test)
        X_te_transf=pd.DataFrame(X_te_transf, columns=X_train.columns)
        
    else: # si classes équilibrées
        
        y_train_rus = y_train
        X_tr=modele[:-1].fit_transform(X_train)
        X_tr_transf=pd.DataFrame(X_tr, columns=X_train.columns)
    
        # Preprocessing y_train
    
        #X_te_transf=modele[:-1].fit_transform(X_test)
        X_te_transf=modele[:-1].transform(X_test)
        
        X_te_transf=pd.DataFrame(X_te_transf, columns=X_train.columns)
    
       
    return X_tr_transf,X_te_transf, y_train_rus

In [ ]:
best_LGBM[0]

In [ ]:
X_train_rus,X_test_sca,y_train_rus=decomposition_modele(best_LGBM)

rus = RandomUnderSampler

X-train-rus, X_test_sca, et y_train_rus contiennent AMNT GOOD PRICES

In [ ]:
print('Train set:',X_train_rus.shape)
print('Test set:',X_test_sca.shape)
print('Train target:',y_train_rus.shape)
print('Test target:',y_test.shape)

#### Avant entrainement du modèle, je retire la feat AMT_GOODS_PRICE
- sur X_train-rus 
- sur X_test scalé

In [ ]:
X_train_rus=X_train_rus.drop(['AMT_GOODS_PRICE'],axis=1)
X_train_rus

In [ ]:
X_test_sca=X_test_sca.drop(['AMT_GOODS_PRICE'],axis=1)
X_test_sca

### Entrainement du modèle

In [ ]:
EST=best_LGBM[-1]
EST

In [ ]:
#EST=LGBMClassifier(max_depth=3, min_data_in_leaf=500,
#                                num_leaves=20)

In [ ]:
EST.fit(X_train_rus, y_train_rus)

In [ ]:
y_pred=EST.predict(X_test_sca)

In [ ]:
y_pred

In [ ]:
y_proba=EST.predict_proba(X_test_sca)
y_proba

In [ ]:
y_proba[:,0]

In [ ]:
#y_proba[:,1]

In [ ]:
np.unique(y_pred, return_counts=True)

In [ ]:
print_score(y_test, y_pred)

In [ ]:
my_specificity_score(y_test,y_pred)

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test,y_pred)).plot()

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test,y_pred)).plot()

In [ ]:
y_pred

### Reconstituion du df X_test avec:
- AMNT_GOODS_PRICE (note  du 22/02/24: AMNT8GOOD_PRICE n'a pa sété retiré de X_test mais de X_te_SCA)
- y_true => y_test
- y_pred => colonne prédictions
- y_proba => en 2 colonnes proba_0 et proba_1 ou une seule colonne sachant que proba_1=1 - proba_0


Ici copier d'abord y_test

In [ ]:
X_test['y_true']=y_test
X_test

In [ ]:
X_test['predictions']=y_pred
X_test

In [ ]:
X_test['proba_0']=y_proba[:,0]
X_test

Ajout de la colonne nb d'annuité qui va nosu permettre de calculer le montant d'interet gagné ou perdu

In [ ]:
X_test['nb_annuité']=round(X_test['AMT_CREDIT']/X_test['AMT_ANNUITY'],0)
X_test['nb_annuité'].describe()

In [ ]:
X_test.loc[(X_test['y_true']==0) & (X_test['predictions']==1),:].head()

X_test_2 ci dessous permet de remettre l'index à 0. 
- Plus facile pour tester les différentes fonction de calcul de gain

In [ ]:
X_test_2=X_test.copy()

In [ ]:
X_test_2.reset_index(inplace=True)

In [ ]:
X_test_2.loc[(X_test_2['y_true']==0) & (X_test_2['predictions']==0),:].head()

## 9.2 Fonctions permettant de calculer le P/L par crédit puis sur l'ensemble du dataset

Ci dessous plusiseurs hypothèses:

1. Dans un 1er calcul prise en compte du manque à gagner (Faux positif) et de perte qui aurait dû être évitées (Faux négatif) => abandonné pour simplifiaction
2. en considérant ou non que les interets ne sont perçus qu'une année ou sur l'ensembel de la durée de vie du crédit => une seule annuité retenue pour simplification

Reflexion sur les différents cas de figure:

- Si y-pred = y_true = 0 => gain= interêts perçus sur le crédit = M * i * n
    
    M= Montant du crédit ;    
    i = taux d'interêt ;    
    n = nb d'année;
        
    Mais pas de gain par rapport à la situation sans modèle de prédiction
    

- Si y-pred = y_true = 1 => Si on considère que le prêt a été accordé à tort dans la réalité et que grace à notre modèle on n'octroie pas le crédit, le gain est une perte évitée soit:

    Montant du crédit moins revente du produit financé (moins décôte de 30%) => M-(30% * PP)

- Si y_pred =1 et y_true= 0 => Faux négatif: montant non accordé à tort: Manque à gagner (perte) = M * i * n

- SI y_pred = 0 et y_true =1 => Faux positif: montant accordé à tort: Perte = Montant du crédit - revente du bien acheté (avec décote de 30% par ex) => M-(30% * PP)

Rappel:  les NaN sur AMNT_GOODS_PRICE ont été défaultés avec le montant du crédit associé (cf nobe explo/transfo: les seuls NaN sont sur des crédits revolving pour lesquels dans la plupart des cas le montant de crédit est égal au prix du bien financé)

In [ ]:
def calcul_gain_unit(ser, lost_coeff =0.3, taux=0.05): #y_true, y_pred
    
    '''Le but de cette fonction est de calculer le différentiel de gain grâce au modèle de prédiction comparé à la réalisation dans la vraie vie'''
        
    if ser['predictions']: # ==1 , pas de prêt
        if ser['y_true']: #==1; on ne perd plus d'argent grace au modèle de prédiction: gain
            return round(ser['AMT_CREDIT']-lost_coeff*ser['AMT_GOODS_PRICE'],2)
        else: # y_true = 0: on a refusé le crédit à tort: manque à gagner = interets non perçus = M*i*n (interets simples)
            return round(-(ser['AMT_CREDIT']*taux*ser['nb_annuité']),2)
    if not ser['predictions']: #==0, prêt accordé (pas de différence avec la réalité)
        return 0 
        #if y_true: #==1, prêt non remboursé, donc prêt accordé à tort=> perte = M-(30% * PP)
        #    return -(ser['AMT_CREDIT']-lost_coeff*ser['AMT_GOODS_PRICE'])
        #elif: # y_true = 0 pas de différence : le gain est le même avec et sans modèle de prédiction
        #    return 0
        
            
        

In [ ]:
def calcul_gain_unit_1an(ser, lost_coeff =0.3, taux=0.05): #y_true, y_pred
    
    '''Variante de la fn calcul_gain_unit où l'on considère une seule annuité'''
        
    if ser['predictions']: # ==1 , pas de prêt
        if ser['y_true']: #==1; on ne perd plus d'argent grace au modèle de prédiction: gain
            return round(ser['AMT_CREDIT']-lost_coeff*ser['AMT_GOODS_PRICE'],2)
        else: # y_true = 0: on a refusé le crédit à tort: manque à gagner = interets non perçus = M*i*n (interets simples)
            return round(-(ser['AMT_CREDIT']*taux),2)
    if not ser['predictions']: #==0, prêt accordé (pas de différence avec la réalité)
        return 0 

Test fonction calcul_gain_unit

In [ ]:
X_test.iloc[19].T

In [ ]:
calcul_gain_unit(X_test.iloc[19].T)

In [ ]:
def calcul_gain_total(df):
    df['Gain']=0
    for i in range(len(df)):
        #calcul_gain_unit(df.iloc[i].T)
        df['Gain'].iloc[i]=calcul_gain_unit(df.iloc[i].T)
    return round(df['Gain'].sum(),2)

Test fonction calcul_gain_total

In [ ]:
calcul_gain_total(X_test)

Montant très négatif. Expliqué par:

- taux de récup et règle basée sur nb d'annuités 
- par le fait que l'on n'a pas les crédits refusés dans la réalité et qui auraient été octroyés grâce au modèle

Si 1 seule annuité

In [ ]:
def calcul_gain_total_1an(df):
    df['Gain']=0
    for i in range(len(df)):
        #calcul_gain_unit(df.iloc[i].T)
        df['Gain'].iloc[i]=calcul_gain_unit_1an(df.iloc[i].T)
    return round(df['Gain'].sum(),2)

In [ ]:
calcul_gain_total_1an(X_test)

Cette fois on gagne

### 9.2.1 Version alternative

pour un crédit donné, calcul du gain ou de la perte, en focntion de la prédictions. Pas de notion de perte évitée ou de manque à gagner

In [ ]:
def calcul_gain_gross(ser, lost_coeff =0.3, taux=0.05): #y_true, y_pred
    
    if ser['predictions']:
        return 0
    
    if not ser['y_true']: # == 0 > prêt accordé et remboursé: gains = interets
        return round(ser['AMT_CREDIT']*taux*ser['nb_annuité'],2)
    
    if ser['y_true']: # ==1 prêt accordé à tort : perte K  - vente des biens
        return round(-ser['AMT_CREDIT']-lost_coeff * ser['AMT_GOODS_PRICE'],2)

Test de la fn calcul_gain_gross

In [ ]:
calcul_gain_gross(X_test.iloc[19].T)

In [ ]:
X_test.iloc[0].T

In [ ]:
calcul_gain_gross(X_test.iloc[0].T)

In [ ]:
def calcul_gain_total_1an(df):
    df['Gain']=0
    for i in range(len(df)):
        #calcul_gain_unit(df.iloc[i].T)
        df['Gain'].iloc[i]=calcul_gain_unit_1an(df.iloc[i].T)
    return round(df['Gain'].sum(),2)

In [ ]:
def calcul_gain_gross_total(df):
    gain=[]
    gain_total = 0
    for i in range(len(df)):
        gain.append(calcul_gain_gross(df.iloc[i].T))
        
        #df['Gain_net'].iloc[i]=calcul_gain_unit(df.iloc[i].T)
        gain_total+=gain[i]
    return round(gain_total,2)

In [ ]:
calcul_gain_gross_total(X_test)

## 9.3 Optimisation du seuil optimal

In [ ]:
def calcul_gain_proba_unit(ser,seuil=0.50, lost_coeff =0.3, taux=0.05):
    
    """Fonction permettant de calculer le gain ou la perte attendus sur un prêt accordé selon qu'il ait été remboursé ou non
    
    ##### METTRE A JOUR fonctions.py de API_V2 #####
    
    
    Args:
    - ser: serie Pandas => colonne 'probe_0' du dataset 
    - seuil: par défaut = 0.5 (seuil de base pour définri la classe 0 ou 1 d'une prédiction)
    -lost_coeff: estimation du taux de revente d'un bien financé par le crédit (utilisé en cas d'impayé)
    - taux: taux d'intéret du crédit
        
    """
    if ser['proba_0'] < seuil:
        return 0    
            
    #if ser['predictions']:
    #    return 0
    
    
    
    if not ser['y_true']: # == 0 > prêt accordé et remboursé: gains = interets
        #Version où les interets sont perçus sur toutes les annuités
        #return round(ser['AMT_CREDIT']*taux*ser['nb_annuité'],2)
        
        #Version où l'on ne perçoit les int qu'une seule fois
        return round(ser['AMT_CREDIT']*taux,2)
    
    if ser['y_true']: # ==1 prêt accordé à tort : perte K  - vente des biens
        return round(-ser['AMT_CREDIT']-lost_coeff * ser['AMT_GOODS_PRICE'],2)

In [ ]:
X_test_2.loc[(X_test_2['y_true']==0) & (X_test_2['predictions']==1),:].head()

In [ ]:
X_test_2.iloc[5].T

In [ ]:
for i in [0.2,0.3,0.4]:
    print(f'gain ={calcul_gain_proba_unit(X_test.iloc[5].T,seuil=i)}')

In [ ]:
range(len(np.arange(0,1.05,0.05)))

In [ ]:
def calcul_seuil_optimal(df):
    
    """ Fonction renvoyant un DataFrame renvoyant pour une série (entre 0 et 1), le gain total attendu.
    DataFrame utilisé pour définir le seuil optimal à partir duquel on considère que le prêt, qu'il ait été accordé ou non, rapportera de l'argent
    Dépend de la fonction calcul_gain_proba_unit définie + haut
    
    ##### METTRE A JOUR fonctions.py de API_V2? #####

    Arg:    
    df: contenant ['proba_0'] (predict proba 0)
    
    """
    
    best_thresh=pd.DataFrame(columns=['threshold','Gain_total'])
    #global best_thresh
    
    for thr,j in zip(np.arange(0,1.05,0.05),range(len(np.arange(0,1.05,0.05)))): 
        gain=[]
        gain_total = 0
        
        for i in range(len(df)):
            gain.append(calcul_gain_proba_unit(df.iloc[i].T,thr,taux=0.04,lost_coeff=0.3))
            gain_total+=gain[i]
        
        tmp=pd.DataFrame([[round(thr,2),gain_total]],columns=['threshold','Gain_total'])
        
        best_thresh=pd.concat([best_thresh,tmp],axis=0,ignore_index=True)
        
    return best_thresh #gain_total

In [ ]:
thresh=calcul_seuil_optimal(X_test)
thresh

In [ ]:
fig = plt.figure(figsize=(6,4),constrained_layout = True)
plt.plot(thresh['threshold'],thresh['Gain_total'], color='teal', label='CA')
#plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
listOf_Xticks = thresh['threshold']
plt.xticks(listOf_Xticks,rotation='vertical')
fig.suptitle('evolution du CA en fn du seuil', fontsize=20)
plt.legend(loc="upper right")
plt.show()

In [ ]:
thresh['Gain_total'].max()

In [ ]:
thresh.loc[thresh['Gain_total']==thresh['Gain_total'].max(),:]

#### Vérification calcul:

In [ ]:
X_test.shape

In [ ]:
X_test.loc[X_test['y_true']==0].shape

In [ ]:
X_test.loc[X_test['y_true']==1].shape

In [ ]:
X_test.loc[(X_test['y_true']==0),'AMT_CREDIT'].sum()*0.04#-X_test.loc[(X_test['y_true']==1),'AMT_GOODS_PRICE'].sum()*0.3

In [ ]:
tmp= X_test.loc[(X_test['y_true']==1),['AMT_GOODS_PRICE','AMT_CREDIT']]

In [ ]:
tmp['AMT_CREDIT'].sum()-0.3*(tmp['AMT_GOODS_PRICE'].sum())

In [ ]:
X_test.loc[(X_test['y_true']==0),'AMT_CREDIT'].sum()*0.04 - tmp['AMT_CREDIT'].sum()-0.3*(tmp['AMT_GOODS_PRICE'].sum())

In [ ]:
thresh.loc[thresh['Gain_total']==thresh['Gain_total'].min(),:]

### Conclusion

Seuil opptimal = 0.7

Point d'amélioration: inclure le calcul du gain max dans la comparaison entre les modèles

# 10 Préparation df pour API

X_test (données sur lesquelles le modèle n'est pas entrainé) duquel je retire les colonnes ajoutées:
- 'y_true',
- 'predictions',
- 'proba_0',
- 'nb_annuité',
- 'Gain'

In [ ]:
X_test=X_test.drop(['y_true','predictions','proba_0','nb_annuité'],axis=1)
X_test.shape

## Modification des entêtes de colonnes pour meilleure lecture dans dashboard

N/A pour le moment: sera géré dans le dashboard via un expander d'explication

## Serialisation pour API

Serialisation depuis pd.to_picle()

In [ ]:
X_test.to_pickle("./data2pkl_V2.pkl")

In [ ]:
ds= open("data2pkl_V2.pkl","rb")
ds_sample=pd.read_pickle(ds)
ds_sample

In [ ]:
ds_sample=ds_sample.sample(frac=0.2)
ds_sample.shape


In [ ]:
ds_sample.to_pickle("./data_smpl.pkl")

In [ ]:
X_test.to_csv('C:/Users/xavie/P7/P7_API/data_sample_V2.csv', index=False)